# Install libraries

In [32]:
!pip install -U pip setuptools wheel
!pip install -U spacy-nightly --pre

[!] Skipping pipeline package dependencies and setting `--no-deps`. You don't
seem to have the spaCy package itself installed (maybe because you've built from
source?), so installing the package dependencies would cause spaCy to be
downloaded, which probably isn't what you want. If the pipeline package has
other dependencies, you'll have to install them manually.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2020-12-12 11:13:36.053324: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-12-12 11:13:36.053357: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Running setup.py clean for fasttext
Failed to build fasttext
    Running setup.py install for fasttext: started
    Running setup.py install for fasttext: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\alex\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Alex\\AppData\\Local\\Temp\\pip-install-rms3ku0v\\fasttext_7cd64d18472b49aea90625c8b8b15d9d\\setup.py'"'"'; __file__='"'"'C:\\Users\\Alex\\AppData\\Local\\Temp\\pip-install-rms3ku0v\\fasttext_7cd64d18472b49aea90625c8b8b15d9d\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Alex\AppData\Local\Temp\pip-wheel-alrjvba_'
       cwd: C:\Users\Alex\AppData\Local\Temp\pip-install-rms3ku0v\fasttext_7cd64d18472b49aea90625c8b8b15d9d\
  Complete output (18 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\fasttext
  copying python\fasttext_module\fasttext\FastText.py -> build\lib.wi

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-win_amd64.whl size=228470 sha256=034f3be04c8611f074e20b1703ea50cda7daacc76bcb0498099437e4a8637f6f
  Stored in directory: c:\users\alex\appdata\local\pip\cache\wheels\93\61\2a\c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [49]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.0.0rc2                      
Location         C:\Users\Alex\AppData\Roaming\Python\Python38\site-packages\spacy
Platform         Windows-10-10.0.19041-SP0     
Python version   3.8.5                         
Pipelines        en_core_web_sm (3.0.0a0), es_core_news_lg (3.0.0a0), es_dep_news_trf (3.0.0a0)



# Import libraries

In [2]:
import pickle
import pandas as pd
import time
import re
import spacy
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import *
pd.set_option('display.max_colwidth', None)

In [3]:
nlp = spacy.load('es_core_news_lg')
import string
punct = string.punctuation
from spacy.lang.es.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

# Loading data

In [4]:
dataset = pd.read_csv('misoginy_classification.csv')

In [5]:
dataset

,full_text,Misoginy
0,Feminazis ni que hijueputas.,1
1,Locas Feminazi aborteras matabebes.,1
2,Ahora me salieron feminazis jajaja,1
3,"Joda aquí en Twitter si debe haber feminazi, menor tengo cuidado con lo que digo.",1
4,"No soy anti lgtb...no me molestan...ni me preocupan, son menos peligrosos que las extremistas feminazis y aborteras.",1
...,...,...
5387,🤣😂😂😂 viral apenas para las feminazis,0
5388,JAJAJAJAJAJAJAJA me hartan las feminazis,0
5389,"Que pena das..., es claro que tu no sabes y no está en tu ADN lo que es amor al prójimo por eso sólo sabes amenazar ""si algún día te pasa algo las feminazis van a pedir justicia por ti""¿quien a pedido justicia por las Carabineras golpeadas, insultadas y quemadas por la 1ra línea?",0
5390,"Les metieron el ganzo ciego máximo con el metro, se han hecho la vista gorda sobre el tema y reniegan de machista a todo el que señale lo obvio\nEso, es un comportamiento feminazi\nA parte, ser mujer no te exenta de la critica y de los errores, niña",0


# WORD2VEC

In [32]:
import fasttext.util
fasttext.util.download_model('es', if_exists='ignore')
ft = fasttext.load_model('cc.es.300.bin')

In [33]:
dataset['full_text'] = dataset['full_text'].apply(lambda x: x.replace('\n',''))

In [34]:
vectorized_dataset = pd.DataFrame(np.array(dataset['full_text'].apply(lambda x: ft.get_sentence_vector(x.strip()))).T.tolist())

In [35]:
vectorized_dataset['Misoginy'] = list(dataset['Misoginy'])

In [36]:
vectorized_dataset

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,Misoginy
0,0.015064,-0.024864,0.076429,0.006004,0.032963,0.013960,0.024403,-0.027771,-0.029196,-0.009106,...,-0.065525,-0.018201,0.023274,-0.000966,-0.010685,-0.007568,-0.022060,0.024691,0.024129,1
1,-0.023828,0.011653,0.112363,0.005050,0.034058,0.006004,0.024562,-0.011510,-0.025849,0.011196,...,-0.053762,-0.122575,-0.028579,0.015134,0.029048,0.024668,0.007888,0.068028,0.022139,1
2,0.010425,0.010731,-0.015589,0.034801,0.003740,0.024220,-0.009103,0.005835,-0.038946,0.042486,...,-0.034128,-0.057281,-0.003371,-0.003159,0.001802,-0.023381,-0.035662,0.022853,-0.000949,1
3,0.008355,-0.018336,0.007510,0.020182,-0.001361,0.005230,0.018139,0.011158,-0.022905,-0.037617,...,-0.088209,-0.024071,0.000450,-0.002201,0.000120,-0.018339,-0.030257,0.008064,0.004792,1
4,0.001572,-0.014950,0.020370,0.011702,0.006239,-0.002801,0.023327,-0.015657,-0.040228,-0.015754,...,-0.070634,-0.073011,0.008080,-0.000280,0.009355,-0.020027,-0.022493,0.007372,-0.003132,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5387,0.004805,0.023890,0.016948,0.025678,0.031679,0.027242,-0.004319,-0.009167,-0.021792,-0.008498,...,-0.015071,-0.031114,0.019297,-0.006607,0.011680,0.004043,-0.024458,0.001019,0.012439,0
5388,-0.006577,0.018284,-0.015334,0.060326,-0.025681,0.025760,-0.025040,0.017250,-0.023369,0.024661,...,-0.019569,-0.050146,0.011798,-0.024951,-0.019259,-0.007067,-0.013879,0.030014,0.012537,0
5389,0.008577,-0.009564,0.007636,0.012107,0.003262,-0.000833,0.022761,-0.012618,-0.021480,-0.046360,...,-0.044840,-0.008009,0.004412,-0.005274,-0.010666,-0.000627,-0.022533,-0.002839,-0.010021,0
5390,-0.003996,-0.007003,-0.001372,0.019508,0.005108,0.012425,0.003185,-0.011791,-0.017455,-0.020266,...,-0.049752,-0.015274,-0.002036,-0.007576,-0.001014,-0.014299,-0.008916,0.000211,0.005622,0


In [130]:
classifier = RandomForestClassifier()
n_estimators = [2000]
max_depth = [1]
min_samples_split = [32]
min_samples_leaf = [24] 
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

model_sel = GridSearchCV(classifier, hyperF,verbose=2,n_jobs=-1,cv =5)

In [131]:
X = vectorized_dataset2.drop('Misoginy',axis=1)
y = vectorized_dataset2['Misoginy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [133]:
model_sel.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.0min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1], 'min_samples_leaf': [24, 25, 26],
                         'min_samples_split': [32, 35, 37],
                         'n_estimators': [2000, 2500, 2250]},
             verbose=2)

In [134]:
y_pred = model_sel.predict(X_test)

In [135]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.56      0.59       508
           1       0.64      0.68      0.66       571

    accuracy                           0.63      1079
   macro avg       0.62      0.62      0.62      1079
weighted avg       0.63      0.63      0.63      1079



In [136]:
model_sel.best_params_

{'max_depth': 1,
 'min_samples_leaf': 26,
 'min_samples_split': 32,
 'n_estimators': 2000}

# Word2Vec XGBOOST

In [119]:
X = vectorized_dataset2.drop('Misoginy',axis=1)
y = vectorized_dataset2['Misoginy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [142]:
classifier = XGBClassifier()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.002,0.0025], #so called `eta` value
              'max_depth': [1],
              'min_child_weight': [16,17],
              'subsample': [0.8],
              'colsample_bytree': [0.7,0.71,0.69],
              'n_estimators': [3500], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}
model_sel = GridSearchCV(classifier, parameters, n_jobs=-1, 
                   cv=5, 
                   scoring='roc_auc',
                   verbose=2, refit=True)

In [143]:
model_sel.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.1min finished
C:\Users\Alex\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:49:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.

In [144]:
y_pred = model_sel.predict(X_test)

In [145]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58       508
           1       0.63      0.64      0.63       571

    accuracy                           0.61      1079
   macro avg       0.61      0.61      0.61      1079
weighted avg       0.61      0.61      0.61      1079



In [146]:
model_sel.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.0025,
 'max_depth': 1,
 'min_child_weight': 16,
 'missing': -999,
 'n_estimators': 3500,
 'nthread': 4,
 'objective': 'binary:logistic',
 'seed': 1337,
 'subsample': 0.8}

# Word2Vec MLP

In [57]:
X = vectorized_dataset.drop('Misoginy',axis=1)
y = vectorized_dataset['Misoginy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [89]:
classifier = MLPClassifier(max_iter=200)
parameter_space = {
    'hidden_layer_sizes': [(100,100,50), (50,100,50), (100,),(50,100,),(50,100,100),(50,100,100,50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.002,0.001,0.0015,0.0018],
    'learning_rate': ['constant','adaptive'],
}
model_sel = GridSearchCV(classifier, parameter_space, n_jobs=-1, 
                   cv=5, 
                   verbose=2, refit=True)

In [90]:
model_sel.fit(X_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 20.1min finished
C:\Users\Alex\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'alpha': [0.002, 0.001, 0.0015, 0.0018],
                         'hidden_layer_sizes': [(100, 100, 50), (50, 100, 50),
                                                (100,), (50, 100),
                                                (50, 100, 100),
                                                (50, 100, 100, 50)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['adam']},
             verbose=2)

In [91]:
y_pred = model_sel.predict(X_test)

In [92]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.56      0.59       508
           1       0.64      0.69      0.66       571

    accuracy                           0.63      1079
   macro avg       0.63      0.63      0.63      1079
weighted avg       0.63      0.63      0.63      1079



In [93]:
model_sel.best_params_

{'activation': 'logistic',
 'alpha': 0.002,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'solver': 'adam'}

# TFIDF Random forest

In [7]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        if token.is_alpha:
            temp = token.lower_.strip()
            tokens.append(temp)
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [12]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = RandomForestClassifier()
params_random = {
    'n_estimators': [4000,6000],   
    'min_samples_split':[16,17],
    'min_samples_leaf' : [12,11,10],
    'min_weight_fraction_leaf' : [0,1],
    'max_depth' : [None,1]
}
model_sel = GridSearchCV(classifier, params_random,cv=5,verbose=2,n_jobs=-1)

In [13]:
X = dataset['full_text']
y = dataset['Misoginy']

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [190]:
clf = Pipeline([('tfidf', tfidf), ('clf', model_sel)])
clf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  4.2min finished


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000001BB073A89D0>)),
                ('clf',
                 GridSearchCV(cv=5, estimator=RandomForestClassifier(),
                              n_jobs=-1,
                              param_grid={'max_depth': [None, 1],
                                          'min_samples_leaf': [12, 11, 10],
                                          'min_samples_split': [16, 17],
                                          'min_weight_fraction_leaf': [0, 1],
                                          'n_estimators': [4000, 6000]},
                              verbose=2))])

In [191]:
y_pred = clf.predict(X_test)

In [192]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.37      0.51       508
           1       0.62      0.92      0.74       571

    accuracy                           0.66      1079
   macro avg       0.72      0.65      0.63      1079
weighted avg       0.71      0.66      0.63      1079



In [193]:
model_sel.best_params_

{'max_depth': None,
 'min_samples_leaf': 12,
 'min_samples_split': 17,
 'min_weight_fraction_leaf': 0,
 'n_estimators': 6000}

In [ ]:
pickle.dump(model_sel,open("random_forest_Grid_search.pkl", "wb"))

# TFIDF XGBOOSTER

In [45]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = XGBClassifier()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.0023,0.0025], #so called `eta` value
              'max_depth': [2,3],
              'min_child_weight': [11,10],
              'subsample': [0.8,0.85,0.75],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}
model_sel = GridSearchCV(classifier, parameters, n_jobs=-1, 
                   cv=3, 
                   verbose=2, refit=True)

In [46]:
X = dataset['full_text']
y = dataset['Misoginy']

In [47]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [48]:
clf = Pipeline([('tfidf', tfidf), ('clf', model_sel)])
clf.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.9min finished
C:\Users\Alex\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Alex\Anaconda3\lib\site-packages\xgboost\data.py:20: UserWarning: `missing` is not used for current input data type:<class 'scipy.sparse.csr.csr_matrix'>
  warnings.warn(


[21:13:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x00000161095B58B0>)),
                ('clf',
                 GridSearchCV(cv=3,
                              estimator=XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta...
                                                      scale_pos

In [49]:
y_pred = clf.predict(X_test)

In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.37      0.50       508
           1       0.62      0.92      0.74       571

    accuracy                           0.66      1079
   macro avg       0.71      0.64      0.62      1079
weighted avg       0.70      0.66      0.63      1079



In [51]:
model_sel.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.0025,
 'max_depth': 3,
 'min_child_weight': 10,
 'missing': -999,
 'n_estimators': 1000,
 'nthread': 4,
 'objective': 'binary:logistic',
 'seed': 1337,
 'subsample': 0.85}

# TFIDF MLP

In [8]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(50,100,50), (100,),(50,100,),(50,100,100),(50,100,100,50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.002,0.001,0.0015,0.0018],
    'learning_rate': ['constant','adaptive'],
}
model_sel = GridSearchCV(classifier, parameter_space, n_jobs=-1, 
                   cv=3, 
                   verbose=2, refit=True)

In [9]:
X = dataset['full_text']
y = dataset['Misoginy']

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [11]:
clf = Pipeline([('tfidf', tfidf), ('clf', model_sel)])
clf.fit(X_train, y_train)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 99.0min finished
C:\Users\Alex\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000001E7A4B33700>)),
                ('clf',
                 GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=100),
                              n_jobs=-1,
                              param_grid={'activation': ['logistic', 'tanh',
                                                         'relu'],
                                          'alpha': [0.002, 0.001, 0.0015,
                                                    0.0018],
                                          'hidden_layer_sizes': [(50, 100, 50),
                                                                 (100,),
                                                                 (50, 100),
                                                                 (50, 100, 100),
                                                                 (50, 100, 100,
                                                                  50

In [12]:
y_pred = clf.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.41      0.41      0.41       508
           1       0.47      0.47      0.47       571

    accuracy                           0.44      1079
   macro avg       0.44      0.44      0.44      1079
weighted avg       0.44      0.44      0.44      1079



In [14]:
model_sel.best_params_

{'activation': 'logistic',
 'alpha': 0.001,
 'hidden_layer_sizes': (50, 100, 100, 50),
 'learning_rate': 'adaptive',
 'solver': 'adam'}